In [ ]:
# My imports
from pylab import *
import numpy as np
import pyfits,glob,time
from subprocess import call
topdir='/Users/cslage/Research/LSST/code/GUI/'
forwarddir ='/Users/cslage/Research/LSST/code/galsim-newgit/GalSim/devel/brighter-fatter/forward_model_varying_i'
%cd $forwarddir
import  forward
from scipy.special import erf
import emcee
from astropy.stats import sigma_clip

%matplotlib inline

In [ ]:
def run_sextractor(fitsfilename):
    configfile=topdir+'sextractor/30-micron-pinholes.sex'
    paramfile=topdir+'sextractor/default-array_dither.param'
    minadu=10
    outname=fitsfilename

    def make_reg_from_ldac(cat_ldac_file,text_tag):
        thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
        f = open(cat_ldac_file+'.reg','w')
        for i in range(len(thecat)):
            xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
            r=thecat['A_IMAGE'][i]
            thetext=thecat[text_tag][i]
            f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
        f.close()
 
    callstring=["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat',"-CATALOG_TYPE","FITS_LDAC",
               "-CHECKIMAGE_NAME",outname[:-4]+"back.fits"+','+outname[:-4]+"rms.fits",
               "-CHECKIMAGE_TYPE","BACKGROUND,BACKGROUND_RMS",
               "-DETECT_THRESH","10","-DETECT_MINAREA","10","-THRESH_TYPE","RELATIVE","-DETECT_MAXAREA","400",
               "-ANALYSIS_THRESH","2"]

    test=call(callstring)
    make_reg_from_ldac(outname+'.cat','NUMBER')



In [ ]:
thedir='/Users/cslage/Research/LSST/code/GUI/spot_sizes/'
filelist=sort(glob.glob(thedir+'bf_1.fits'))
print len(filelist)," files"


In [ ]:
tstart=time.time()
for i in range(len(filelist)): run_sextractor(filelist[i])

print time.time()-tstart

In [ ]:
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.x2=zeros([nstamps])
        self.y2=zeros([nstamps])
        self.xy=zeros([nstamps])
        self.imax=zeros([nstamps])

def BuildSpotListFromFits(fitsfilename, numspots, nx, ny, minsize, maxsize):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    xcoomin = 0
    xcoomax = 200
    ycoomin = 0
    ycoomax = 200
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    hdr=pyfits.getheader(fitsfilename)
    img=pyfits.getdata(fitsfilename) 
    catname=fitsfilename[:-5]+'.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    catfile.close()
    n=0
    for line in catlines:
        try:
            size = float(line.split()[3].split('#')[0])
            if size < minsize or size > maxsize:
                continue
            xcoord = float(line.split()[1])
            ycoord = float(line.split()[2])
            if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                continue
            xint = int(xcoord-0.5)
            yint = int(ycoord-0.5)
            xmin = xint - int(nx/2)
            xmax = xint + int(nx/2) + 1
            ymin = yint - int(ny/2)
            ymax = yint + int(ny/2) + 1
            stamp = img[ymin:ymax, xmin:xmax]
           
            xsum = 0.0
            ysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    ysum += spotlist.y[j] * spotlist.data[i,j,n]
                    xsum += spotlist.x[i] * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.xoffset[n] = xoff
            spotlist.yoffset[n] = yoff

            x2sum = 0.0
            y2sum = 0.0
            xysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    x2sum += (spotlist.x[i] - spotlist.xoffset[n]) * (spotlist.x[i] - spotlist.xoffset[n]) * spotlist.data[i,j,n]
                    y2sum += (spotlist.y[j] - spotlist.yoffset[n]) * (spotlist.y[j] - spotlist.yoffset[n]) * spotlist.data[i,j,n]
                    xysum += (spotlist.x[i] - spotlist.xoffset[n]) * (spotlist.y[j] - spotlist.yoffset[n]) * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.x2[n] = x2sum / datasum
            spotlist.y2[n] = y2sum / datasum
            spotlist.xy[n] = xysum / datasum
                    
            n += 1
            if n == numspots:
                return spotlist
        except:
            continue
    # Reaching this point means we found less spots than requested.
    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,n)
    newspotlist.xoffset = spotlist.xoffset[0:n]
    newspotlist.yoffset = spotlist.yoffset[0:n]
    newspotlist.x2 = spotlist.x2[0:n]
    newspotlist.y2 = spotlist.y2[0:n]
    newspotlist.xy = spotlist.xy[0:n]
    newspotlist.data = spotlist.data[:,:,0:n]
    del spotlist
    return newspotlist


    spotlist = Array2dSet(stampxmin,stampxmax,Nx,stampymin,stampymax,Ny,NumSpots)
    for spot in range(NumSpots):
        filename = dirbase+'run_%d/%s_%d_CC.dat'%(spot,outputfilebase,run)
        elec = ReadCCFile(filename, Nx, Ny)
        cfgfile = dirbase+'run_%d/pixel.cfg'%spot
        SpotConfigData = ReadConfigFile(cfgfile)
        xsum = 0.0
        ysum = 0.0
        elecsum = 0.0
        for i in range(Nx):
            for j in range(Ny):
                spotlist.data[i,j,spot] = elec[i,j]
                elecsum += elec[i,j]
                xsum = spotlist.x[i] * elec[i,j]
                ysum = spotlist.y[j] * elec[i,j]
        spotlist.xoffset[spot] = xsum / elecsum
        spotlist.yoffset[spot] = ysum / elecsum

def ReadCCFile(filename, Nx, Ny):
    file = open(filename,'r')
    lines=file.readlines()
    file.close()
    lines.remove(lines[0]) # Strip the title line
    elec = zeros([Nx,Ny])
    count = 0
    for line in lines:
        items = line.split()
        i = int(items[0]) - 1
        j = int(items[1]) - 1        
        elec[i,j] = int(items[2])
    return elec


def BuildSpotListFromCCFiles(dirbase, outputfilebase, run, numspots, nx, ny):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    for spot in range(numspots):
        filename = dirbase+'run_%d/%s_%d_CC.dat'%(spot,outputfilebase,run)
        elec = ReadCCFile(filename, nx, ny)
        xsum = 0.0
        ysum = 0.0
        elecsum = 0.0
        for i in range(nx):
            for j in range(ny):
                spotlist.data[i,j,spot] = elec[i,j]
                elecsum += elec[i,j]
                xsum += spotlist.x[i] * float(elec[i,j])
                ysum += spotlist.y[j] * float(elec[i,j])
        spotlist.xoffset[spot] = xsum / float(elecsum)
        spotlist.yoffset[spot] = ysum / float(elecsum)
    return spotlist


def lnprob(sigma, spotlist, sigma_min, sigma_max):
    result = forward.forward(spotlist,sigma,sigma)
    if isnan(log(result)): return -1000.0
    elif sigma >= sigma_min and sigma <= sigma_max:
        return -log(result)
    elif sigma < sigma_min :
        result = forward.forward(spotlist,sigma_min,sigma_min)
        return -log(result) - 1000.0 * (sigma_min - sigma)
    elif sigma > sigma_max :
        result = forward.forward(spotlist,sigma_max,sigma_max)
        return -log(result) - 1000.0 * (sigma - sigma_max)
    else:
        return -1000.0



In [ ]:
# We build the list of spots with 9x9 postage stamps
spotlist = BuildSpotListFromFits(filelist[0], 100, 9, 9, 1.3, 1.7)
# Let's look at a random spot
print spotlist.nstamps
imshow(spotlist.data[:,:,33],interpolation='Nearest')

In [ ]:
# Now use emcee to fit the spots
nwalkers = 250
ndim = 1
p0 = np.random.rand(ndim * nwalkers).reshape((nwalkers, ndim))
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=[spotlist, 0.5, 2.5])
pos, prob, state = sampler.run_mcmc(p0, 100)
sampler.reset()
sampler.run_mcmc(pos, 1000)

In [ ]:
print("Mean acceptance fraction: {0:.3f}"
                .format(np.mean(sampler.acceptance_fraction)))
clipped_chains = sigma_clip(np.ma.masked_outside(sampler.flatchain[:,0],1.0,2.0), 4)
print clipped_chains.mean(), clipped_chains.std(), clipped_chains.count()
figure()
title("MCMC GalSim Spot Size Calculation")
bins = linspace(1.3,1.7,100)
hist(clipped_chains, bins, color="k")
text(1.26, 30000, "Mean Sigma = %.5f"%(clipped_chains.mean()), color = 'red') 
text(1.26, 27000, "Std Sigma = %.5f"%(clipped_chains.std()), color = 'red') 
text(1.26, 24000, "N = %d"%(clipped_chains.count()), color = 'red') 
savefig(thedir+"MCMC_GalSim_Spots_11Apr18.pdf")


In [ ]:
# Re-running with 1024 spots
dirbase = base = '/Users/cslage/Research/LSST/code/poisson/Poisson_CCD_Hole20_Fe55/data/llnl_fe55_14/cc_files/'
outputfilebase = 'Pixel'
spotlist = BuildSpotListFromCCFiles(dirbase, outputfilebase, 0, 1024, 5, 5)
# Let's look at a random spot
n=118
print spotlist.nstamps
imshow(spotlist.data[:,:,n],interpolation='Nearest')
print spotlist.data[:,:,n]
print spotlist.xoffset[n], spotlist.yoffset[n]

In [ ]:
print lnprob(0.30,spotlist,0.05,1.5)

In [ ]:
# Now use emcee to fit the spots
nwalkers = 250
ndim = 1

p0 = 0.35 + 0.2 *np.random.rand(ndim * nwalkers).reshape((nwalkers, ndim))

sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=[spotlist, 0.05, 1.5])
pos, prob, state = sampler.run_mcmc(p0, 100)
sampler.reset()
sampler.run_mcmc(pos, 1000)

In [ ]:
print("Mean acceptance fraction: {0:.3f}"
                .format(np.mean(sampler.acceptance_fraction)))
clipped_chains = sigma_clip(np.ma.masked_outside(sampler.flatchain[:,0],0.0,0.8), 4)
print clipped_chains.mean(), clipped_chains.std(), clipped_chains.count()
figure()
title("MCMC Sigma Calculation - Run14 - 1024 Events")
bins = linspace(0.0,0.8,100)
hist(clipped_chains, bins, color="k")
text(0.5, 10000, "Mean Sigma = %.3f"%(clipped_chains.mean()), color = 'red') 
text(0.5, 9000, "Std Sigma = %.3f"%(clipped_chains.std()), color = 'red') 
text(0.5, 8000, "N = %d"%(clipped_chains.count()), color = 'red') 
savefig(thedir+"MCMC_Sigma_Run14_11Apr18.pdf")


In [ ]:
# Re-running with 1024 spots
dirbase = base = '/Users/cslage/Research/LSST/code/poisson/Poisson_CCD_Hole20_Fe55/data/llnl_fe55_14/cc_files/'
outputfilebase = 'Pixel'
spotlist = BuildSpotListFromCCFiles(dirbase, outputfilebase, 0, 1024, 5, 5)
xaxis = linspace(0.10,0.40,1000)
lprob = array([lnprob(x,spotlist,0.10,0.40) for x in xaxis])
plot(xaxis, lprob)

In [ ]:
# Re-running with 128 spots
dirbase = base = '/Users/cslage/Research/LSST/code/poisson/Poisson_CCD_Hole20_Fe55/data/llnl_fe55_14_small/cc_files/'
outputfilebase = 'Pixel'
spotlist = BuildSpotListFromCCFiles(dirbase, outputfilebase, 0, 128, 5, 5)
xaxis = linspace(0.10,0.40,1000)
lprob = array([lnprob(x,spotlist,0.10,0.40) for x in xaxis])
plot(xaxis, lprob)

In [ ]:
def lnprob2(sigmax, sigmay, spotlist):
    result = forward.forward(spotlist,sigmax,sigmay)
    return -log(result)


# Re-running with 128 spots
dirbase = base = '/Users/cslage/Research/LSST/code/poisson/Poisson_CCD_Hole20_Fe55/data/llnl_fe55_14_small/cc_files/'
outputfilebase = 'Pixel'
spotlist = BuildSpotListFromCCFiles(dirbase, outputfilebase, 0, 128, 5, 5)
xaxis = linspace(0.10,0.40,200)
yaxis = linspace(0.10,0.40,200)
[yy,xx] = meshgrid(yaxis,xaxis)
lprob = zeros([200,200])
for i,x in enumerate(xaxis):
    for j,y in enumerate(yaxis):
        lprob[i,j] = lnprob2(x,y,spotlist)
contourf(xx, yy, lprob)

In [ ]:
from scipy.optimize import fmin_powell

def FOM(params):
    [sigmax, sigmay] = params
    result = forward.forward(spotlist,sigmax,sigmay)
    return result


# Re-running with 128 spots
dirbase = base = '/Users/cslage/Research/LSST/code/poisson/Poisson_CCD_Hole20_Fe55/data/llnl_fe55_14_small/cc_files/'
outputfilebase = 'Pixel'
spotlist = BuildSpotListFromCCFiles(dirbase, outputfilebase, 0, 128, 5, 5)

param0 = [1.0, 1.0]
args = ()
Result = fmin_powell(FOM, param0, args)
print Result

param0 = [0.30, 0.30]
args = ()
Result = fmin_powell(FOM, param0, args)
print Result

def FOM1(params):
    [sigma] = params
    result = forward.forward(spotlist,sigma,sigma)
    return result

param0 = [0.30]
args = ()
Result = fmin_powell(FOM1, param0, args)
print Result




In [ ]:
# Why is the different from the 0.35 that was calculated on LLNL?????